<a href="https://colab.research.google.com/github/petteriTeikari/minivess_mlops/blob/main/notebooks/train_custom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use a custom model quickly

Reuse the data(loader) from the "ML pipe"

If you run this Notebook on Colab, you need to install the Virtual Environment with Poetry yourself (what I understood):

In [1]:
import sys
import os

running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    print('You are running this on COLAB so installing the environment here')
    # TODO! Colab by default uses Python 3.10 so force the 3.8.17 here?
    os.chdir("/content")
    !git clone https://github.com/petteriTeikari/minivess_mlops.git
    !pip install poetry
    os.chdir("/content/minivess_mlops")
    !poetry config virtualenvs.in-project true
    # Running "!poetry install --no-ansi" is needed or not?
    !poetry install
    #!poetry shell
    # https://stackoverflow.com/a/65440080/6412152
    !pip install dvc
    !pip install dvc-s3
else:
    print('Assuming that you are runnign this from IDE,\n'
          'or some other environment where you have your Jupyter kernel created from Poetry files')

You are running this on COLAB so installing the environment here
fatal: destination path 'minivess_mlops' already exists and is not an empty directory.
Installing dependencies from lock file

No dependencies to install or update
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 827.5 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of fsspec[http] to determine which version is compatible with other requirements. This could take a while.
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
    Uninstalling fsspec-2023.6.0:
      Successfully uninstalled fsspec-2023.6.0
ERROR: pip's dependency resolver does not currently 

Debugger for Colab, https://zohaib.me/debugging-in-google-collab-notebook/

In [2]:
!pip install -Uqq ipdb
import ipdb
%pdb on

Automatic pdb calling has been turned ON


Import the other modules now available from Poetry environment and from the Github repo (see [in_Google_Colab.ipynb](https://github.com/elise-chin/poetry-and-colab/blob/main/Using_python_poetry_in_Google_Colab.ipynb))

In [3]:
!poetry show -v # /content/minivess_mlops/.venv

Using virtualenv: /content/minivess_mlops/.venv
absl-py                  1.4.0        Abseil Python Common Libraries, see https://github.com/ab...
aiobotocore              2.7.0        Async client for aws services using botocore and aiohttp
aiohttp                  3.8.6        Async http client/server framework (asyncio)
aiohttp-retry            2.8.3        Simple retry client for aiohttp
aioitertools             0.11.0       itertools and builtins for AsyncIO and mixed iterables
aiosignal                1.3.1        aiosignal: a list of registered asynchronous callbacks
alembic                  1.12.0       A database migration tool for SQLAlchemy.
amqp                     5.1.1        Low-level AMQP client for Python (fork of amqplib).
annotated-types          0.6.0        Reusable constraint types to use with typing.Annotated
antlr4-python3-runtime   4.9.3        ANTLR 4.9.3 runtime for Python 3.7
appdirs                  1.4.4        A small Python module for determining appropr

In [4]:
if running_in_colab:
  VENV_BASE = '/content/minivess_mlops/.venv'
  VENV_PATH = os.path.join(VENV_BASE, 'lib', 'python3.10', 'site-packages')
  #VENV_PATH = "/content/gdrive/MyDrive/test-poetry/.venv/lib/python3.8/site-packages"
  #!mkdir /content/venv
  LOCAL_VENV_PATH = '/content/venv_poetry' # local notebook
  try:
    os.symlink(VENV_PATH, LOCAL_VENV_PATH) # connect to directory in drive
  except Exception as e:
    print('symlink failed, e = {}'.format(e))
  sys.path.insert(0, LOCAL_VENV_PATH)

  sys.path.insert(0,'/content/minivess_mlops')

symlink failed, e = [Errno 17] File exists: '/content/minivess_mlops/.venv/lib/python3.10/site-packages' -> '/content/venv_poetry'


In [5]:
from loguru import logger

from src.run_training import parse_args_to_dict
from src.training.experiment import define_experiment_data
from src.utils.config_utils import import_config

/content/venv_poetry/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/content/venv_poetry/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?


Import helper subfunction(s)

In [6]:
def get_dataloaders(experim_dataloaders: dict):
    # Get the "validation" and "train" dataloaders from the dictionary
    fold_name = 'fold0'
    split_names = list(experim_dataloaders[fold_name].keys())
    fold_key = experim_dataloaders.get(fold_name)
    if fold_key is not None:
        try:
            train = experim_dataloaders[fold_name]['TRAIN']
            val = experim_dataloaders[fold_name]['VAL']['MINIVESS']
        except Exception as e:
            raise IOError('Could not get the dataloaders from the dictionary, error = {}'.format(e))
    else:
        raise IOError('Fold name = "{}" not found in the dataloaders dictionary'.format(fold_name))
    return train, val

Input arguments for the training (you can add all the input arguments supported by `run_training.py` here

In [7]:
input_args = ['-c', 'tutorials/train_demo']

# Fake these as coming from the command line to match the main code (run_training.py)
sys.argv = ['notebook_run']  # Jupyter has all the extra crap, so replace that with this
for sysargv in input_args:
    sys.argv.append(sysargv)
args = parse_args_to_dict()

2023-11-02 01:24:50.221 | INFO     | src.run_training:parse_args_to_dict:73 - Parsed input arguments:
2023-11-02 01:24:50.225 | INFO     | src.utils.general_utils:print_dict_to_logger:43 - task_config_file: tutorials/train_demo
2023-11-02 01:24:50.228 | INFO     | src.utils.general_utils:print_dict_to_logger:43 - run_mode: train
2023-11-02 01:24:50.232 | INFO     | src.utils.general_utils:print_dict_to_logger:43 - data_dir: /mnt/minivess-dvc-cache
2023-11-02 01:24:50.234 | INFO     | src.utils.general_utils:print_dict_to_logger:43 - output_dir: /mnt/minivess-artifacts
2023-11-02 01:24:50.238 | INFO     | src.utils.general_utils:print_dict_to_logger:43 - s3_mount: False
2023-11-02 01:24:50.242 | INFO     | src.utils.general_utils:print_dict_to_logger:43 - local_rank: 0
2023-11-02 01:24:50.246 | INFO     | src.utils.general_utils:print_dict_to_logger:43 - project_name: minivess-test2


Create the config with Hydra from the .yaml file(s)

In [8]:
from src.utils.config_utils import config_import_script, set_up_experiment_run

config = config_import_script(args['task_config_file'],
                              parent_dir_string = '.', #f'..{os.sep}..',
                              parent_dir_string_defaults = f'..{os.sep}..')
print(config)

2023-11-02 01:24:51.356 | INFO     | src.utils.config_utils:hydra_import_config:91 - Initializing Hydra with config_path = "/content/minivess_mlops/./configs", job_name = "config_test", version_base = "1.2"
2023-11-02 01:24:51.364 | INFO     | src.utils.config_utils:hydra_import_config:93 - Hydra overrides list = ['+tutorials=train_demo']


{'VERSION': '0.01alpha', 'NAME': 'train_demo', 'config': {'VERBOSE': True, 'DATA': {'DATA_SOURCE': {'DATASET_NAMES': ['MINIVESS'], 'FOLDER': {'PLACEHOLDER': 'placeholder', 'TRANSFORMS': {'TEST': 'NO_AUG'}, 'DATASET': {'NAME': 'MONAI_CACHEDATASET', 'MONAI_CACHEDATASET': {'CACHE_RATE': 0.0, 'NUM_WORKERS': 1}}}, 'MINIVESS': {'FETCH_METHOD': 'DVC', 'FETCH_METHODS': {'DVC': {'datapath_base': 'data', 'version': 'placeholder'}, 'EBrains': {'DATA_INFO_URL': 'https://doi.org/10.25493/HPBE-YHK', 'DATA_DOWNLOAD_URL': 'https://data.kg.ebrains.eu/zip?container=https://data-proxy.ebrains.eu/api/v1/buckets/d-bf268b89-1420-476b-b428-b85a913eb523'}}, 'SPLITS': {'NAME': 'RANDOM', 'RANDOM': {'SEED': 42, 'TEST_VAL_RATIO': 0.1, 'TRAIN_RATIO': 0.8}, 'TXT_FILE': {'PLACEHOLDER': 0}}, 'SUBSET': {'NAME': 'ALL_SAMPLES', 'ALL_SAMPLES': {'PLACEHOLDER_PARAM': 'placeholder'}, 'DEBUG': {'TRAIN': 2, 'VAL': 2, 'TEST': 1}}, 'DATASET': {'NAME': 'MONAI_CACHEDATASET', 'MONAI_CACHEDATASET': {'CACHE_RATE': 0.0, 'NUM_WORKERS'

In [9]:
if running_in_colab:
  if os.path.exists('/mnt'):
    os.makedirs(args['data_dir'], exist_ok=True)  # DVC Cache
    os.makedirs(args['output_dir'], exist_ok=True)  # Artifacts output


Authenticate to AWS S3?
* [Public read-only s3 remote](https://discuss.dvc.org/t/public-read-only-s3-remote/355/3)
* [manage_data.ipynb - Colaboratory](https://colab.research.google.com/github/khuyentran1401/Efficient_Python_tricks_and_tools_for_data_scientists/blob/master/Chapter5/manage_data.ipynb)
* [DagsHub integration with Google Colab](https://dagshub.com/docs/integration_guide/google_colab/)

Pull the data from DVC

In [10]:
!dvc pull

ERROR: failed to connect to s3 (minivessdataset/files/md5) - Unable to locate credentials
Fetching
ERROR: failed to pull data from the cloud - 1 files failed to download


In [11]:
cfg = import_config(args=args, task_cfg_name=args['task_config_file'])

2023-11-02 01:24:53.372 | ERROR    | src.utils.config_utils:get_config_path:37 - Configuration dir "/content/minivess_mlops/../configs" does not exist!


OSError: ignored

Import the dataloaders (now the data augmentations are here as well as data transformations)

In [ ]:
_, _, experim_dataloaders, cfg['run'] = define_experiment_data(cfg=cfg)

# Get the "validation" and "train" dataloaders from the dictionary
train, val = get_dataloaders(experim_dataloaders)

Now you are ready to train your new model that you just wanna quickly test without
wanting to have a battle with the config .YAML files

Add maybe some fastai demo with MLflow autologging:
[https://github.com/mlflow/mlflow/blob/master/examples/fastai/train.py](https://github.com/mlflow/mlflow/blob/master/examples/fastai/train.py)

In [ ]:
# Iterate the dataloaders for demo
no_of_epochs = 3
logger.info('Training for {} epochs'.format(no_of_epochs))
for epoch in range(no_of_epochs):

    logger.info('Epoch {}/{}'.format(epoch, no_of_epochs - 1))

    # Train
    logger.info('train with {} batches'.format(len(train)))
    for i, batch in enumerate(train):
        images, mask = batch['image'], batch['label']

    # Validation
    logger.info('validate with {} batches'.format(len(train)))
    for j, batch in enumerate(val):
        images, mask = batch['image'], batch['label']

logger.info('Training done!')